# Title: AI Agent for traveler

## objective: search information about the location user defined.
### information: 
        1. route
        2. whether report

### to ask user 
        1,2. location
        3. when
        4. purpose
        5. favorite route

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [3]:
from google.adk.agents import Agent
# from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
# from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import AgentTool, FunctionTool, google_search,ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
#### Helper function
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [5]:
# retry configuration: 
retry_config = types.HttpRetryOptions(
    attempts=3,  # Maximum retry attempts
    exp_base=3,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors

)

In [6]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent.
    Your job is to use the google_search to find the following two information and answer in compacted and readable style for human.
    1. travel method by route from point_of_departure to destination for purpose.
    2. wheather report at the time on the destination.
    """,
    tools=[google_search],
    # output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
    output_key="travel_route"
)

print("✅ research_agent created.")

✅ research_agent created.


In [7]:
# Check Inputted Location valifity

Check_agent = Agent(
    name = "CheckAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized check agent.
    Your only job is to use the google_search to verify user input is exiting city or county on the Earth.
    And if it is 'True', answer should be 'location in Country'. e.g.,'Tokyo in Japan'. 
     else, aswer should be 'False'in boolean response.""",
    tools=[google_search],
    output_key="cheked_asking",
)

print("✅ check_agent created.")

✅ check_agent created.


In [8]:
def get_asking(point_of_departure = "Haneda",destination = "",when = "tommorrow", purpose = "sightseen",route = "air") -> str:
    user_asking = ('I\'m planing a travel from ' + point_of_departure + ' to ' + destination +
              '. The travel is planning at ' + when +     
              '. The purpose of the travel is ' + purpose + '. I would like to do the travel by ' + route +
              '. But If it is less practical, I would like to do the travel by air.')
    print(user_asking)
    return user_asking

In [9]:
# Check Inputted Location
def check_location(location) -> str:
    # Location list
    locations ={"Tokyo" : "Japan", "Saitama" : "Japan", "London":"United Kingdum","Gerlin":"Germany","New York":"United States"}
    if locations.get(location):
        ans =input(f"You mean {location} in {locations.get(location)}? y/n")
        if ans in ("y","Y","yes","YES","Yes"):
            return 'True'
        else:
            return 'False'
    
        

In [10]:
# set up runner
travel_runner = InMemoryRunner(agent=research_agent)
check_runner = InMemoryRunner(agent=Check_agent)

In [12]:
# main
print("I'm an assistant for your travel planing.")
s = input("Where is the point of departure? >>")

if check_location(s):
    pass
else:
    response = await check_runner.run_debug(s)
    if 'False' in response[0].content.parts[0].text:
        print("I don't know the lacation. I assume it would be Tokyo.")
        s = "Tokyo"

d = input("Where will you go? >>")

if check_location(d):
    pass
else:
    response = await check_runner.run_debug(d)
    if 'False' in response[0].content.parts[0].text:
        print("I don't know the lacation. I assume it would be London")
        d = "London"

t = input("When will you go? >> ")
p = input("What is your purpose? >>")
f = input("Do you have favourite route air, train, ship, etc. >>")
user_asking =get_asking(s,d,t,p,f)
response = await travel_runner.run_debug(user_asking)
# print(response)

I'm an assistant for your travel planing.


Where is the point of departure? >> london



 ### Created new session: debug_session_id

User > london
CheckAgent > London in United Kingdom.


Where will you go? >> paris



 ### Continue session: debug_session_id

User > paris
CheckAgent > Paris in France.


When will you go? >>  next april
What is your purpose? >> visiting friends
Do you have favourite route air, train, ship, etc. >> ferry


I'm planing a travel from london to paris. The travel is planning at next april. The purpose of the travel is visiting friends. I would like to do the travel by ferry. But If it is less practical, I would like to do the travel by air.

 ### Continue session: debug_session_id

User > I'm planing a travel from london to paris. The travel is planning at next april. The purpose of the travel is visiting friends. I would like to do the travel by ferry. But If it is less practical, I would like to do the travel by air.
ResearchAgent > For your trip from London to Paris next April to visit friends, traveling by ferry is feasible but less practical than flying.

**Travel by Ferry:**
There is no direct ferry service from London to Paris. You would need to take a train from London to a ferry port such as Dover, then take a ferry to a French port like Calais or Dieppe, and finally travel by train or other means to Paris. The entire journey by train and ferry can take around 11 hours, and may be m

### End of The Code ###